# Transfer Learning

In this notebook, you'll learn how to use pre-trained networks to solved challenging problems in computer vision. Specifically, you'll use networks trained on [ImageNet](http://www.image-net.org/) [available from torchvision](http://pytorch.org/docs/0.3.0/torchvision/models.html). 

ImageNet is a massive dataset with over 1 million labeled images in 1000 categories. It's used to train deep neural networks using an architecture called convolutional layers. I'm not going to get into the details of convolutional networks here, but if you want to learn more about them, please [watch this](https://www.youtube.com/watch?v=2-Ol7ZB0MmU).

Once trained, these models work astonishingly well as feature detectors for images they weren't trained on. Using a pre-trained network on images not in the training set is called transfer learning. Here we'll use transfer learning to train a network that can classify our cat and dog photos with near perfect accuracy.

With `torchvision.models` you can download these pre-trained networks and use them in your applications. We'll include `models` in our imports now.

In [0]:
# Prepare notebook to run on Google Colab environment
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-1.0.0-{platform}-linux_x86_64.whl torchvision

!pip install Pillow==4.1.1

!wget https://s3.amazonaws.com/content.udacity-data.com/nd089/Cat_Dog_data.zip
!unzip Cat_Dog_data.zip

In [0]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models

Most of the pretrained models require the input to be 224x224 images. Also, we'll need to match the normalization used when the models were trained. Each color channel was normalized separately, the means are `[0.485, 0.456, 0.406]` and the standard deviations are `[0.229, 0.224, 0.225]`.

In [0]:
data_dir = 'Cat_Dog_data'

# DONE: Define transforms for the training data and testing data
train_transforms = transforms.Compose([transforms.RandomHorizontalFlip(p=0.3),
                                       transforms.RandomVerticalFlip(p=0.3),
                                       transforms.RandomRotation(45),
                                       transforms.RandomResizedCrop(224,scale=(0.5,1.0),ratio=(1.,1.)),
                                       transforms.ToTensor()])

test_transforms = transforms.Compose([transforms.RandomResizedCrop(224,scale=(0.5,1.0),ratio=(1.,1.)),
                                      transforms.ToTensor()])

# Pass transforms in here, then run the next cell to see how the transforms look
train_data = datasets.ImageFolder(data_dir + '/train', transform=train_transforms)
test_data = datasets.ImageFolder(data_dir + '/test', transform=test_transforms)

trainloader = torch.utils.data.DataLoader(train_data, batch_size=125, shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=90, shuffle=True)

We can load in a model such as [DenseNet](http://pytorch.org/docs/0.3.0/torchvision/models.html#id5). Let's print out the model architecture so we can see what's going on.

In [0]:
model = models.densenet121(pretrained=True)
model

This model is built out of two main parts, the features and the classifier. The features part is a stack of convolutional layers and overall works as a feature detector that can be fed into a classifier. The classifier part is a single fully-connected layer `(classifier): Linear(in_features=1024, out_features=1000)`. This layer was trained on the ImageNet dataset, so it won't work for our specific problem. That means we need to replace the classifier, but the features will work perfectly on their own. In general, I think about pre-trained networks as amazingly good feature detectors that can be used as the input for simple feed-forward classifiers.

In [0]:
# Freeze parameters so we don't backprop through them
for param in model.parameters():
    param.requires_grad = False

from collections import OrderedDict
classifier = nn.Sequential(OrderedDict([
                          ('fc1', nn.Linear(1024, 500)),
                          ('relu', nn.ReLU()),
                          ('fc2', nn.Linear(500, 2)),
                          ('output', nn.LogSoftmax(dim=1))
                          ]))
    
model.classifier = classifier

With our model built, we need to train the classifier. However, now we're using a **really deep** neural network. If you try to train this on a CPU like normal, it will take a long, long time. Instead, we're going to use the GPU to do the calculations. The linear algebra computations are done in parallel on the GPU leading to 100x increased training speeds. It's also possible to train on multiple GPUs, further decreasing training time.

PyTorch, along with pretty much every other deep learning framework, uses [CUDA](https://developer.nvidia.com/cuda-zone) to efficiently compute the forward and backwards passes on the GPU. In PyTorch, you move your model parameters and other tensors to the GPU memory using `model.to('cuda')`. You can move them back from the GPU with `model.to('cpu')` which you'll commonly do when you need to operate on the network output outside of PyTorch. As a demonstration of the increased speed, I'll compare how long it takes to perform a forward and backward pass with and without a GPU.

In [0]:
import time

In [7]:
for device in ['cpu', 'cuda']:

    criterion = nn.NLLLoss()
    # Only train the classifier parameters, feature parameters are frozen
    optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)

    model.to(device)

    for ii, (inputs, labels) in enumerate(trainloader):

        # Move input and label tensors to the GPU
        inputs, labels = inputs.to(device), labels.to(device)

        start = time.time()

        outputs = model.forward(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        if ii==3:
            break
        
    print(f"Device = {device}; Time per batch: {(time.time() - start)/3:.3f} seconds")

Device = cpu; Time per batch: 15.071 seconds
Device = cuda; Time per batch: 0.016 seconds


You can write device agnostic code which will automatically use CUDA if it's enabled like so:
```python
# at beginning of the script
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

...

# then whenever you get a new Tensor or Module
# this won't copy if they are already on the desired device
input = data.to(device)
model = MyModule(...).to(device)
```

From here, I'll let you finish training the model. The process is the same as before except now your model is much more powerful. You should get better than 95% accuracy easily.

>**Exercise:** Train a pretrained models to classify the cat and dog images. Continue with the DenseNet model, or try ResNet, it's also a good model to try out first. Make sure you are only training the classifier and the parameters for the features part are frozen.

In [0]:
# Use CUDA if it is available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [0]:
## DONE: Use a pretrained model to classify the cat and dog images
model = models.resnet152(pretrained=True)
model

In [0]:
# Freeze all parameters
for param in model.parameters():
    param.requires_grad = False

In [11]:
# Create a new classifier
classifier = nn.Sequential(nn.Linear(2048,1024),
                           nn.ReLU(),
                           nn.Dropout(0.2),
                           nn.Linear(1024,2),
                           nn.LogSoftmax(dim = 1))
model.fc = classifier
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=F

In [0]:
# Define criterion and optimizer
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.fc.parameters())

In [13]:
# Build the train loop
epochs = 1
print_every = 10

for e in range(epochs):
    # Set model to train mode
    model.train()
    running_loss = 0
    for step, (images, labels) in enumerate(trainloader):
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        output = model(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

        if (step+1) % print_every == 0:
            # Set model to evaluation mode
            model.eval()
            # turn off gradients
            with torch.no_grad():
                output_sum = 0
                testing_loss = 0
                accuracy = 0
                for images, labels in testloader:
                    images, labels = images.to(device), labels.to(device)

                    # Make inferences
                    output = model(images)

                    # Calculate the loss
                    loss = criterion(output, labels)
                    testing_loss += loss.item()

                    # Get the most likely classes
                    ps = torch.exp(output)
                    top_p, top_class = ps.topk(1, dim = 1)

                    # Calculate the accuracy
                    equals = top_class == labels.view(*top_class.shape)
                    accuracy += torch.mean(equals.type(torch.FloatTensor))

                else:
                    running_loss /= print_every
                    testing_loss /= len(testloader)
                    accuracy /= len(testloader)

                    print(f'Epoch: {e+1}/{epochs}\t' +
                          f'Training loss: {running_loss:.4f}\t' +
                          f'Test loss: {testing_loss:.4f}\t' +
                          f'Accuracy: {accuracy.item()*100:.2f}%')
                    
                    running_loss = 0
                    model.train()

Epoch: 1/1	Training loss: 0.8939	Test loss: 0.2674	Accuracy: 89.43%
Epoch: 1/1	Training loss: 0.3492	Test loss: 0.1132	Accuracy: 97.05%
Epoch: 1/1	Training loss: 0.1941	Test loss: 0.0755	Accuracy: 97.79%
Epoch: 1/1	Training loss: 0.1590	Test loss: 0.0659	Accuracy: 97.82%
Epoch: 1/1	Training loss: 0.1691	Test loss: 0.0869	Accuracy: 96.81%
Epoch: 1/1	Training loss: 0.1844	Test loss: 0.0508	Accuracy: 98.36%
Epoch: 1/1	Training loss: 0.1353	Test loss: 0.0458	Accuracy: 98.37%
Epoch: 1/1	Training loss: 0.1397	Test loss: 0.0485	Accuracy: 98.40%
Epoch: 1/1	Training loss: 0.1418	Test loss: 0.0404	Accuracy: 98.43%
Epoch: 1/1	Training loss: 0.1274	Test loss: 0.0418	Accuracy: 98.77%
Epoch: 1/1	Training loss: 0.1401	Test loss: 0.0492	Accuracy: 98.49%
Epoch: 1/1	Training loss: 0.1456	Test loss: 0.0470	Accuracy: 98.57%
Epoch: 1/1	Training loss: 0.1066	Test loss: 0.0526	Accuracy: 98.28%
Epoch: 1/1	Training loss: 0.1090	Test loss: 0.0540	Accuracy: 98.25%
Epoch: 1/1	Training loss: 0.1522	Test loss: 0.04